# Zarr format and HDF5 format read time comparison 
Compute the maximum water level during Hurricane Ike on a 9 million node triangular mesh storm surge model.  This reads 53GB of data. 

In [ ]:
import xarray as xr
import zarr
import fsspec
import fsspec.implementations.reference as refs
import intake
import intake_xarray

In [ ]:
import xarray;print(xarray.__version__)
import zarr;print(zarr.__version__)
import s3fs;print(s3fs.__version__)
import fsspec;print(fsspec.__version__)
import intake;print(intake.__version__)
import intake_xarray;print(intake_xarray.__version__)
import aiobotocore;print(aiobotocore.__versions__)

### Start a dask cluster to crunch the data

In [ ]:
from dask.distributed import Client

In [ ]:
from dask_gateway import Gateway
gateway = Gateway()
cluster = gateway.new_cluster()

In [ ]:
cluster.scale(30);

In [ ]:
cluster

In [ ]:
client = Client(cluster)

### Open Intake Catalog

In [ ]:
cat = intake.open_catalog('intake_catalog.yml')
list(cat)

### Zarr library reading HDF5 file with fsspec

In [ ]:
ds  = cat['ike-hdf5'].to_dask()

In [ ]:
%%time
# 30-worker cluster
max_var = ds['zeta'].max(dim='time').compute()

### Zarr library reading Zarr format data

In [ ]:
ds  = cat['ike-zarr'].to_dask()

In [ ]:
ds.time[0]

In [ ]:
import pandas as pd

In [ ]:
start = pd.to_datetime("2008-09-05 12:00:00")  #2008090512 from Clint 2018-06-

Compute the max water level at each grid cell (reads all the data):

In [ ]:
%%time
max_var1 = ds['zeta'].max(dim='time').compute()

### compare a point from both datasets, should be the same

In [ ]:
max_var[1000].values

In [ ]:
max_var1[1000].values